# 1.screening

In [ ]:
import os, pandas as pd, numpy as np # importar funciones externas
import anexo_3_udfs as mf            # importar funciones propias (UDFS)

print(os.getcwd())                   # comprobar WD

mf.viz()                             # ampliar visualizaciones de pandas

### importar dataframes

In [ ]:
# last re-check: repetir lo mismo para df entero
input_path = 'Anexo_3_carpeta_1_inputs/tabla_3.3.1_articulos_para_screening.csv'
df = pd.read_csv(input_path)
abs_vc_df = df["Abstract Note"].isna().value_counts(dropna=False)
print(f"Nº de artículos sin abstract (en todo el df): {abs_vc_df}\n")

# 2.screening

***
##### SETTING KEYWORDS & SCREENING

***
CHECKBOX COL ANALYSIS

In [ ]:
# importar dataframe
path = "Anexo_3_carpeta_2_resultados/tabla_3.4_screening_wip.csv"
cdf = pd.read_csv(path, sep=";")

# columnas checkbox
cols = ["ADPKD?", "OMICS?", "BIOINFO?", "HUMAN DATA?", "BIOMARKER?", "TRASLATIONAL?", "DECISION", "COMMENT"]

In [ ]:
i = 0
c = cols[i+1]
cdf[c].value_counts(dropna=False)

In [ ]:
i +=1
c = cols[i]
cdf[c].value_counts(dropna=False)

In [ ]:
i += 1
c = cols[i]
cdf[c].value_counts(dropna=False)

In [ ]:
i += 1
c = cols[i]
cdf[c].value_counts(dropna=False)

In [ ]:
i += 1
c = cols[i]
list(cdf[c].value_counts(dropna=False).index)

In [ ]:
i += 1
c = cols[i]
list(cdf[c].value_counts(dropna=False).index)

In [ ]:
i += 1
c = cols[i]
cdf[c].value_counts(dropna=False)

In [ ]:
i += 1
c = cols[i]
list(cdf[c].value_counts(dropna=False).index)

***
### reclasificar comentarios de screening

De una lista vasta y desorganizada pasamos a una lista condensada que nos permita decidir cuáles repasar

In [ ]:
# importar 
from anexo_3_dicts_resultados import dudosos_comments

# Crear un diccionario invertido (para que .map funcione, ya que busca claves y asigna valores y enfermedad_dict tenía la estructura al revés)
reclasif = {}
for coment, dudas in dudosos_comments.items():
    for duda in dudas:
        reclasif[duda] = coment

# mapear y comprobar
cdf["MOTIVO"] = cdf["COMMENT"].map(reclasif)
cdf[["DECISION", "MOTIVO"]].value_counts(dropna=False)

In [ ]:
# Ahora los dudosos que no repasemos se irán a exclude
c1 = cdf["MOTIVO"] != "Repasar"
c2 = cdf["DECISION"] == "dudoso"
cdf.loc[c1 & c2, "DECISION"] = "exclude"
cdf[["DECISION", "MOTIVO"]].value_counts(dropna=False)

***
### re-screen dudosos

los revisaré manualmente uno por uno

In [ ]:
c2 = cdf["DECISION"] == "dudoso"
dudosos = cdf.loc[c2]
print(dudosos.shape)
col = "DOI"
ds = list(dudosos[col].value_counts().index)
ds

In [ ]:
col = "Title"
list(dudosos[col].value_counts().index)

In [ ]:
col = "COMMENT"
list(dudosos[col].value_counts().index)

In [ ]:
## revisaré los artículos manualmente y cambio sus valores de DECISION y MOTIVO

# artículo 1
i = 0
c = dudosos["DOI"] == ds[i]
dudosos.loc[c, "DECISION"] = "include"
dudosos.loc[c, "MOTIVO"] = np.nan

# artículo 2
i += 1
c = dudosos["DOI"] == ds[i]
dudosos.loc[c, "DECISION"] = "exclude"
dudosos.loc[c, "MOTIVO"] = "Tienen un subgrupo de pacientes ADPKD (o pueden tenerlo), pero no queda clara su aportación"

# artículo 3
i += 1
c = dudosos["DOI"] == ds[i]
dudosos.loc[c, "DECISION"] = "include"
dudosos.loc[c, "MOTIVO"] = np.nan

# artículo 4
i += 1
c = dudosos["DOI"] == ds[i]
dudosos.loc[c, "DECISION"] = "include"
dudosos.loc[c, "MOTIVO"] = np.nan

# artículo 5
i += 1
c = dudosos["DOI"] == ds[i]
dudosos.loc[c, "DECISION"] = "include"
dudosos.loc[c, "MOTIVO"] = np.nan

# reasignar decision en cdf
c2 = cdf["DECISION"] == "dudoso"
cdf.loc[c2, "DECISION"] = "include"

# 3.dividir cdf
in included, exlcuded, dudoso (y guardar)

In [ ]:
cdf.shape

In [ ]:
cdf["DECISION"].value_counts()

In [ ]:
group = "exclude"
c = cdf["DECISION"] == group
df = cdf.loc[c].copy()
print(df.shape)
path = f'Anexo_3_carpeta_2_resultados/tabla_3.5.2_{group}.csv'
df.to_csv(path)

In [ ]:
# motivos de exclusión
cols = ["DECISION"]
df[cols].value_counts()

In [ ]:
group = "include"
c = cdf["DECISION"] == group
df = cdf.loc[c].copy()
df = df.drop_duplicates(subset="DOI")
print(df.shape)
path = f'Anexo_3_carpeta_2_resultados/tabla_3.5.1_{group}.csv'
df.to_csv(path)

# 4.actualizar stats

In [ ]:
c = cdf["DECISION"] == "exclude"
df = cdf.loc[c].copy()
vc = df["MOTIVO"].value_counts()

ex_mots = vc.reset_index()
ex_mots.columns = ["MOTIVO", "count"]

# Añadir como fila 0 el total
temp_df = {"MOTIVO": ["Excluidos: tiab screening"],
           "count": [sum(ex_mots["count"])]}
# pasar a df (y trasponer)
temp_df = pd.DataFrame(temp_df)
temp_df["count"] = temp_df["count"].astype(int)
# concateno los dos
ex_mots = pd.concat([temp_df, ex_mots])
# renombro filas de ex_mots con 1ª col
ex_mots.index = ex_mots["MOTIVO"]
ex_mots = ex_mots.iloc[:,1]
ex_mots = pd.DataFrame(ex_mots)
ex_mots.columns = ["Excluidos"]

# importar stats y concatenarla a ex_mots
stats = pd.read_csv('Anexo_3_carpeta_1_inputs/tabla_3.3.2.stats.csv', index_col=0)
stats = pd.concat([stats, ex_mots])

# hacer restas correspondientes
stats.iloc[-5, 3] = stats.iloc[-6, 3] - stats.iloc[-5, 2]
stats

In [ ]:
# exportar
stats.to_csv('Anexo_3_carpeta_2_resultados/tabla_3.5.3_stats_2.csv', index=True)
print("Generado 'stats.csv' para screening")
print(f"Shape: {stats.shape}")